In [24]:
import pandas as pd
import matplotlib.pyplot as plt

## Project :Network traffic analysis


# Data traffic dataset

In [25]:
#read dataset and load it into dataframe
data = "resources/output.csv"
df = pd.read_csv(data, delimiter=',')

# Exploratory analysis of the dataset

In [26]:
#dataset dimension
shape = df.shape
print(f"Columns: {shape[1]}, Rows: {shape[0]}")

#Remove null data or clean the dataset of garbage elements
df = df.dropna()
print(f"Dataset dimension with no null or bad data \nColumns: {df.shape[1]}, Rows: {df.shape[0]}\n")

Columns: 7, Rows: 585462
Dataset dimension with no null or bad data 
Columns: 7, Rows: 585395



In [27]:
#Data frame summary
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 585395 entries, 0 to 585461
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   No.          585395 non-null  int64  
 1   Time         585395 non-null  float64
 2   Source       585395 non-null  object 
 3   Destination  585395 non-null  object 
 4   Protocol     585395 non-null  object 
 5   Length       585395 non-null  int64  
 6   Info         585395 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 35.7+ MB


In [28]:
# unique values of the 'Protocol' column in the DataFrame df.
print(f"Values of the 'Protocol' \n{df['Protocol'].unique()}")

Values of the 'Protocol' 
['ICMP' 'IPv4' 'DHCPv6' 'ICMPv6' 'ARP' 'DNS' 'TCP' 'SSLv3' 'HTTP' 'TELNET'
 'UDP' 'FTP' 'Messenger' 'MANOLITO' 'THRIFT' 'Gnutella' 'HTTP/XML'
 'IGMPv2' 'IGMPv1' 'RGMP' 'STP' 'CDP' 'Syslog' 'PPP LCP' 'L2TP' 'TLSv1'
 'OCSP' 'RADIUS' 'WSP' 'DHCP' 'PPPoED' 'PPP CHAP' 'PPP IPCP' 'PPP IPV6CP'
 'NTP' 'SIP' 'PPP PAP' 'SIP/SDP' 'RTP' 'Portmap' 'MOUNT' 'NFS' 'RPC'
 'PTPv2' 'SMB2' 'SMTP' 'SMTP/IMF' 'BROWSER' 'SNMP' 'UFTP' 'IGMPv3'
 'FTP-DATA' 'MDNS' 'TLSv1.2' 'SSDP' 'TLSv1.3' 'QUIC' 'HTTP/JSON'
 'PKIX-CRL' 'XMPP/XML' 'LLMNR' 'NNTP' 'SSL' 'SSLv2' 'NBNS']


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
 
from pylab import rcParams
 
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
 
from collections import Counter

# Data preprocessing

# Data transformation


# Dataset Consolidation